# Cvm Daily Funds Data

In [1]:
from catalog_api.infrastructure.CkanApi import CkanApi
import pandas as pd
from datetime import datetime, date
from io import BytesIO
from catalog_api.infrastructure.S3Bucket import S3Bucket

bucket = S3Bucket()
portal = CkanApi("https://dados.cvm.gov.br")

fi_doc_inf_diario = portal.get_package("fi-doc-inf_diario")


In [ ]:
def process_resource(resource):
    url = resource.url
    filename = url.split("/")[-1]
    filename_prefix = filename.split(".")[0]
    partition_full = filename_prefix.split("_")[-1]
    year = partition_full[:4]
    month = partition_full[4:6]
    if not year.isnumeric() or not month.isnumeric():
        print(f"Invalid partition: {filename_prefix}")
        return
    bucket_file = (
        f"catalog-data/br_gov_cvm/fi-doc-inf_diario/{year}/{month}/data.parquet"
    )
    bucket_last_updated = bucket.get_last_updated_date_of_file(bucket_file) or datetime(
        2000, 1, 1
    )
    bucket_last_updated = bucket_last_updated.isoformat()
    resource_last_updated = datetime.fromisoformat(
        resource.created or resource.last_modified
    ).isoformat()
    if bucket_last_updated >= resource_last_updated:
        return
    df = pd.read_csv(url, sep=";", encoding="latin1", low_memory=False)
    df["DT_COMPTC"] = df["DT_COMPTC"].apply(
        lambda x: datetime.strptime(x, "%Y-%m-%d").date() if x else None
    )
    parquet_bytes = BytesIO()
    df.to_parquet(parquet_bytes, index=False)
    parquet_bytes.seek(0)
    bucket.put_bytes(bucket_file, parquet_bytes.getvalue())
    print(f"Uploaded {bucket_file} to S3")


for resource in fi_doc_inf_diario.resources:
    process_resource(resource)

Invalid partition: meta_inf_diario_fi


TypeError: can't compare offset-naive and offset-aware datetimes